#**Import Modules**

Selenium, Openpyxl, Regex, Files, Time

In [28]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2,
         "download_restrictions": 3}
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (122 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [0]:
from selenium.common.exceptions import *
from selenium.webdriver.common.action_chains import ActionChains

In [0]:
import time

In [0]:
import openpyxl

In [0]:
import regex as re

In [0]:
from google.colab import files

#**Mount Google Drive**

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [8]:
cd drive/My Drive/Colab Notebooks/Paper/Trade

/content/drive/My Drive/Colab Notebooks/Paper/Trade


In [9]:
ls

Crawler/  Data/  Paper-Authors/  Paper-Keywords/


#**Fetch all Paper Links**

In [11]:
MAINURL = "https://www.kci.go.kr/kciportal/po/search/poArtiSearList.kci?sereId=000599"
articleUrls = []

# access main page
wd.get(MAINURL)
time.sleep(0.5)

# fetch last page number
try:
    LASTPAGENO = wd.find_element_by_xpath("//a[@href]/img[@alt='마지막']").find_element_by_xpath("..").get_attribute("href")
    LASTPAGENO = int(LASTPAGENO[LASTPAGENO.index("(") + 1:LASTPAGENO.index(")")])
except NoSuchElementException:
    print("NO LAST PAGE!")

# fetch article urls
currPage = 1

while currPage <= LASTPAGENO:
    print("Reading page : " + str(currPage))

    # fetch all articles
    articles = wd.find_elements_by_xpath("//table[@class='list_paper no_title']/tbody/tr/td/p/label/a[@href]")
    for article in articles:
        articleUrls.append(article.get_attribute("href"))

    # go to next page
    try:
        TARGET = "//div[@class='pagenate']/a[@href=\'javascript:goPage(" + str(currPage + 1) + ")\']"
        wd.find_element_by_xpath(TARGET).click()
        currPage += 1
        time.sleep(0.8)
    except NoSuchElementException:
        break

len(articleUrls)

Reading page : 1
Reading page : 2
Reading page : 3
Reading page : 4
Reading page : 5
Reading page : 6
Reading page : 7
Reading page : 8
Reading page : 9
Reading page : 10
Reading page : 11
Reading page : 12
Reading page : 13
Reading page : 14
Reading page : 15
Reading page : 16
Reading page : 17
Reading page : 18
Reading page : 19
Reading page : 20
Reading page : 21
Reading page : 22
Reading page : 23
Reading page : 24
Reading page : 25
Reading page : 26
Reading page : 27
Reading page : 28
Reading page : 29
Reading page : 30
Reading page : 31
Reading page : 32
Reading page : 33
Reading page : 34
Reading page : 35
Reading page : 36
Reading page : 37
Reading page : 38
Reading page : 39
Reading page : 40


398

#**Fetch and Write Data from each Volumes**

In [0]:
wb = openpyxl.Workbook()
ws = wb.active

In [0]:
TARGETPATH = "./Data/raw/Korean/"

In [35]:
 r = 2

 # write header
 ws.cell(row = 1, column = 1).value = 'journal'
 ws.cell(row = 1, column = 2).value = 'year'
 ws.cell(row = 1, column = 3).value = 'title'
 ws.cell(row = 1, column = 4).value = 'keywords'
 ws.cell(row = 1, column = 5).value = 'authors'

 for articleurl in articleUrls:
    # access volume page
    wd.get(articleurl)
    time.sleep(1)

    # fetch title
    try:
        title = wd.find_element_by_xpath("//p[@class='title']/span").get_attribute("innerText")
    except NoSuchElementException:
        continue
    #print(title)

    # fetch year
    try:
        yr = wd.find_element_by_xpath("//div[@class='journalInfo']/p[@class='vol']/a").get_attribute("innerText").strip()
        yr = yr[:yr.index(",")]
    except ValueError:
        continue
    except NoSuchElementException:
        continue
    #print(yr)
        
    # fetch authors
    authors = []
    a_tags = wd.find_elements_by_xpath("//div[@class='author']/a")
    for a in a_tags:
        author = a.get_attribute("innerText").strip()
        author = author[:author.index(" ")]
        authors.append(author)
    while ' ' in authors:
        authors.remove(' ')
    while '' in authors:
        authors.remove('')
    if(len(authors) == 0):
        continue
    #print(authors)

    # fetch keywords
    keywords = []
    try:
        h2_tag = wd.find_element_by_xpath("//div[@class='articleBody']/div[@class='box']/h2[contains(text(), '키워드')]")
        try:
            p_tag = h2_tag.find_element_by_xpath("../div/p")
            try:
                a_tags = h2_tag.find_elements_by_xpath("../div/div/a")
                keywords_candid_1 = p_tag.get_attribute("innerText").strip().split(", ")
                keywords_candid_2 = []
                for a in a_tags:
                    keywords_candid_2.append(a.get_attribute("innerText"))
                candid_1 = 0
                candid_2 = 0
                for kwd in keywords_candid_1:
                    if len(re.compile("[가-힣]+").findall(kwd)) > 0:
                        candid_1 += 1
                for kwd in keywords_candid_2:
                    if len(re.compile("[가-힣]+").findall(kwd)) > 0:
                        candid_2 += 1
                if candid_1 <= candid_2:
                    keywords = keywords_candid_1
                else:
                    keywords = keywords_candid_2
            except NoSuchElementException:
                keywords = p_tag.get_attribute("innerText").strip().split(", ")
        except NoSuchElementException:
            a_tags = h2_tag.find_elements_by_xpath("../div/div/a")
            for a in a_tags:
                keywords.append(a.get_attribute("innerText"))
    except NoSuchElementException:
        continue
    while ' ' in keywords:
        keywords.remove(' ')
    while '' in keywords:
        keywords.remove('')
    '''
    hanguel_idx = []
    for i in range(len(keywords)):
        if len(re.compile("[가-힣]+").findall(keywords[i])) > 0:
            hanguel_idx.append(i)
    hanguel_idx.reverse()
    for i in hanguel_idx:
        del keywords[i]
    '''
    if(len(keywords) == 0):
        continue
    #print(keywords)

    # write data to excel file
    print("Writing, " + title)
    ws.cell(row = r, column = 1).value = "Journal of International Trade and Industry Studies"
    ws.cell(row = r, column = 2).value = yr
    ws.cell(row = r, column = 3).value = title
    for j in range(len(keywords)):
        ws.cell(row = r + j, column = 4).value = keywords[j]
    for j in range(len(authors)):
        ws.cell(row = r + j, column = 5).value = authors[j]
    r += max([len(keywords), len(authors)])

    if articleUrls.index(articleurl) % 10 == 0:
        print(str(articleUrls.index(articleurl) + 1) + " out of " + str(len(articleUrls)))

Writing, 기업의 해외생산 가능성이 세무조사 정책에 미치는 영향
1 out of 398
Writing, 한·인도 CEPA와 여타 FTA가 한·인도 양자무역에 미치는 영향 분석
Writing, 동아시아 소비재무역의 결정요인
Writing, 한-아세안 GVC 무역의 특성과 시사
Writing, 중국기업의 해외 M&A 현황과 시사점
Writing, 한·중·일 FTA 전략적 대응방안과 한계점에 관한 연구
Writing, 리쇼어링 정책이 고용에 미치는 영향
Writing, How does Korea’s Aid for Trade affect bilateral export performance in recipients’ countries?
Writing, Taking a Bigger Slice of the Global Value Chain Pie: An Industry-Level Analysis
Writing, EU의 개정 반덤핑 규정에 대한 법적 분석
Writing, 글로벌 가치사슬 참여와 비교우위
11 out of 398
Writing, 한･인도 CEPA의 품목별 원산지규정 특징과 활용률 분석
Writing, 중국 진출기업의 투자국 다각화 결정요인 분석
Writing, TBT와 SPS 조치가 할랄식품 수출에 미치는 영향
Writing, EU 개인정보 역외이전 규제의 GATS 양립성이 국내 상호주의 법안에 주는 정책 시사점
Writing, 산업연관분석을 통한 중국, 베트남, 캄보디아 산업구조 비교 연구
Writing, 중국 수출품의 가격과 품질 변화 분석
Writing, 무역기술장벽과 EU시장 진입전략
Writing, Impacts of SPS and TBT measures on Korean Agricultural Exports: An Analysis by Disaggregated Products
Writing, 반덤핑 조사에서의 PMS 판정- 우리 무역구제 법령 및 제도에 대한 함의
Writing, 지식재산권 보호를 위한 국경조치 제도의 개선방안 연구
21 out

#**Save File**

In [0]:
wb.save(filename = TARGETPATH + "journal_of_international_trade_and_industry_studies.xlsx")

#**Download FIle**

In [0]:
files.download(TARGETPATH + "journal_of_international_trade_and_industry_studies.xlsx")